### Setup:

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import streamlit as st

In [2]:
NUM_COLS = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

# # load the model
# model = tf.keras.models.load_model("./saved_model.h5")

# dummy code / Incorrect Comment this out!
model = tf.keras.models.Sequential([tf.keras.layers.Dense(units=10, input_shape=(10,))])
model.compile(loss='mse', optimizer='adam')

with open("ohe_scaler.pkl", 'rb') as f:
    ohe, scaler = pickle.load(f)

THRESHOLD = np.array([0.000051])

# what are these objects?
type(model), type(ohe), type(scaler), THRESHOLD

(tensorflow.python.keras.engine.sequential.Sequential,
 dict,
 pandas.core.frame.DataFrame,
 array([5.1e-05]))

### Dummy Data for model inference:

In [3]:
'''
What do the features mean?

1. type: Type of Transaction. Can be one of ['CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER']
2. amount: amount of transaction. Can only be positive.
3. oldbalanceOrg: Initial Balance of the person initiating the transaction.
4. newbalanceOrig: Final Balance of the person initiating the transaction, after money has been transfered.
5. oldbalanceDest: Initial Balance of the person to whom the transaction is addressed.
6. newbalanceDest: Final Balance of the person to whom transacion is addressed, after money has been transfered.

'''

# an example of fraudulant transaction
fraud = {'type': 'CASH_OUT', 'amount': 1619313, 'oldbalanceOrg': 1619313,
         'newbalanceOrig': 0.0, 'oldbalanceDest': 0.0, 'newbalanceDest': 1619313,
         'isFraud': 1}

# an example of non fraudulant transaction
non_fraud = {'type': 'CASH_OUT', 'amount': 1000, 'oldbalanceOrg': 5000,
             'newbalanceOrig': 4000, 'oldbalanceDest': 100, 'newbalanceDest': 1100,
             'isFraud': 0}

# how does it look?
data = pd.DataFrame(data=[fraud, non_fraud])
data

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,CASH_OUT,1619313,1619313,0.0,0.0,1619313,1
1,CASH_OUT,1000,5000,4000.0,100.0,1100,0


### Dummy Model Prediction:

In [4]:
# scale, one_hot encode and make predictions with our model
temp = pd.concat([
    data['type'].apply(lambda x: pd.Series(ohe[x])),
    ((data.iloc[:, 1:-1] - scaler.loc['Mean']) / scaler.loc['Std'])[NUM_COLS]
], axis=1).values

# lets make predictions on our samples
data['Reconstruction_loss'] = np.mean(np.square(np.subtract(
    temp, model.predict(temp))), axis=1)

# make the boolean predictions
data['Predictions'] = data['Reconstruction_loss'] > THRESHOLD[0]

# how does it look? Wrong predictions appear in Red
data.style.apply(lambda x: ['background-color: green' if x['Predictions'] == x['isFraud'] 
                            else 'background-color: red'] * len(x), axis=1)

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,Reconstruction_loss,Predictions
0,CASH_OUT,1619313,1619313,0.000000,0.000000,1619313,1,1.192539,True
1,CASH_OUT,1000,5000,4000.000000,100.000000,1100,0,0.172169,True


### Streamlit inference Logic:

In [5]:
def predict(ttype, amt, org_old_bal, org_new_bal, des_old_bal, des_new_bal, model):
    'Code logic to predict using the model. Assumes data is already preprocessed'
    
    result = []
    
    # scale and one hot encode before feeding to model
    ttype = ohe[ttype]
    amt = (amt - scaler.loc['Mean', 'amount']) / scaler.loc['Std', 'amount']
    org_old_bal = (org_old_bal - scaler.at['Mean', 'oldbalanceOrg']) / scaler.at['Std', 'oldbalanceOrg']
    org_new_bal = (org_new_bal - scaler.at['Mean', 'newbalanceOrig']) / scaler.at['Std', 'newbalanceOrig']
    des_old_bal = (des_old_bal - scaler.at['Mean', 'oldbalanceDest']) / scaler.at['Std', 'oldbalanceDest']
    des_new_bal = (des_new_bal - scaler.at['Mean', 'newbalanceDest']) / scaler.at['Std', 'newbalanceDest']
    
    # stack as a numpy array
    temp = np.stack([*ttype, amt, org_old_bal, org_new_bal, des_old_bal, des_new_bal], axis=0)
    temp = temp[np.newaxis]

    # lets make predictions on our samples
    result.append(np.mean(np.square(np.subtract(temp, model.predict(temp))), axis=1))

    # make the boolean predictions
    result.append(result[0] > THRESHOLD[0])

    return result

In [6]:
def main():
    # front end elements of the web page
    html_temp = """
    <div style ="background-color:yellow;padding:13px">
    <h1 style ="color:black;text-align:center;">Streamlit Credit Card Fraud Detection App</h1>
    </div>
    """

    # display the front end aspect
    st.markdown(html_temp, unsafe_allow_html=True)

    # following lines create boxes in which user can enter data required to make prediction
    ttype = st.selectbox('ttype', ('CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER'))
    amt = st.number_input("Transaction Input")
    org_old_bal = st.number_input("Old Balance of Sender")
    org_new_bal = st.number_input("New Balance of Sender")
    des_old_bal = st.number_input("Old Balance of Reciepient")
    des_new_bal = st.number_input("New Balance of Reciepient")

    result = ""

    # when 'Predict' is clicked, make the prediction and store it
    if st.button("Predict"):
        result = predict(ttype, amt, org_old_bal, org_new_bal, des_old_bal, des_new_bal, model)
        st.success(f'The Transaction is {"fraudulant" if result[1] else "genuine"}!' + 
                   f'Reconstruction Loss: {float(result[0]):.6}')

if __name__ == "__main__":
    NUM_COLS = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

    # # load the model
    # model = tf.keras.models.load_model("./saved_model.h5")

    # dummy code / Incorrect Comment this out!
    model = tf.keras.models.Sequential([tf.keras.layers.Dense(units=10, input_shape=(10,))])
    model.compile(loss='mse', optimizer='adam')

    with open("ohe_scaler.pkl", 'rb') as f:
        ohe, scaler = pickle.load(f)

    THRESHOLD = np.array([0.000051])

    main()

In [7]:
import subprocess
from pyngrok import ngrok

# execute in background
p = subprocess.Popen(["streamlit", "run", "app.py"])

# lets now host it to a temporary public url
public_url = ngrok.connect("8501")
print ("Connect to Public Url:", public_url)

2021-01-28 16:04:23.806 INFO    pyngrok.ngrok: Opening tunnel named: http-8501-ceba422a-aa3d-45d1-835a-586f95561135
2021-01-28 16:04:23.848 INFO    pyngrok.process.ngrok: t=2021-01-28T16:04:23+0530 lvl=info msg="no configuration paths supplied"
2021-01-28 16:04:23.854 INFO    pyngrok.process.ngrok: t=2021-01-28T16:04:23+0530 lvl=info msg="using configuration at default config path" path=/home/kael/.ngrok2/ngrok.yml
2021-01-28 16:04:23.857 INFO    pyngrok.process.ngrok: t=2021-01-28T16:04:23+0530 lvl=info msg="open config file" path=/home/kael/.ngrok2/ngrok.yml err=nil
2021-01-28 16:04:23.863 INFO    pyngrok.process.ngrok: t=2021-01-28T16:04:23+0530 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2021-01-28 16:04:24.839 INFO    pyngrok.process.ngrok: t=2021-01-28T16:04:24+0530 lvl=info msg="tunnel session started" obj=tunnels.session
2021-01-28 16:04:24.840 INFO    pyngrok.process.ngrok: t=2021-01-28T16:04:24+0530 lvl=info msg="client session established" obj=csess id=25

Connect to Public Url: NgrokTunnel: "http://c222754414e5.ngrok.io" -> "http://localhost:8501"


2021-01-28 16:04:25.347 INFO    pyngrok.process.ngrok: t=2021-01-28T16:04:25+0530 lvl=info msg="started tunnel" obj=tunnels name=http-8501-ceba422a-aa3d-45d1-835a-586f95561135 addr=http://localhost:8501 url=https://c222754414e5.ngrok.io
2021-01-28 16:04:25.349 INFO    pyngrok.process.ngrok: t=2021-01-28T16:04:25+0530 lvl=info msg=end pg=/api/tunnels id=abdfc4afbdbc8e74 status=201 dur=484.332897ms
2021-01-28 16:04:25.355 INFO    pyngrok.process.ngrok: t=2021-01-28T16:04:25+0530 lvl=info msg=start pg="/api/tunnels/http-8501-ceba422a-aa3d-45d1-835a-586f95561135 (http)" id=e6be0d9762de517e
2021-01-28 16:04:25.356 INFO    pyngrok.process.ngrok: t=2021-01-28T16:04:25+0530 lvl=info msg=end pg="/api/tunnels/http-8501-ceba422a-aa3d-45d1-835a-586f95561135 (http)" id=e6be0d9762de517e status=200 dur=192.683µs
2021-01-28 16:04:41.200 INFO    pyngrok.process.ngrok: t=2021-01-28T16:04:41+0530 lvl=info msg="join connections" obj=join id=449f9bfb5be0 l=[::1]:8501 r=27.5.237.242:50394
2021-01-28 16:04:4

In [ ]:
# when you wish to terminate do
p.terminate()
ngrok.disconnect(public_url)
ngrok.kill()